# reading tables from berg 2012

[link](https://iopscience.iop.org/article/10.1088/0004-637X/754/2/98#apj433966t4)

In [2]:
import numpy as np
import pandas as pd
import astropy

# Table 1

In [130]:
raw_df = pd.read_csv('data/apj433966t1_ascii.txt', header = None, skipfooter = 6, skiprows = 6, sep = '\t+')
columns = ['Galaxy', 'RA', 'DEC', 'F_4.5', 'F_K_s', 'D', 'Ref', 'M_B', 'M_4.5', 'M_K_s', 'B-V', 'log(L_4.5)', 'log(M_star)']
raw_df.columns = columns

/var/folders/g_/kxb_65m159995chtyg4q96440000gn/T/ipykernel_74613/227386655.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  raw_df = pd.read_csv('data/apj433966t1_ascii.txt', header = None, skipfooter = 6, skiprows = 6, sep = '\t+')


In [131]:
clean_df = raw_df[['Galaxy', 'RA', 'DEC']].copy()

cols_to_split = ['F_K_s']

#first, for cols w upper limits 
for key in cols_to_split:
    temp_df = raw_df[key].str.split(' ', expand = True).copy()
    
    value_temp_df = temp_df[0].str.split('<', expand = True).replace(to_replace='', value=np.nan).astype(float)
    clean_df[key] = value_temp_df[0]
    ul_key = f'{key}_upper_limit'
    clean_df[ul_key] = value_temp_df[1]

    err_key = f'{key}_err'
    clean_df[err_key] = temp_df[2].replace(to_replace='', value=np.nan).astype(float)

cols_to_split = ['D', 'M_K_s', 'log(L_4.5)', 'log(M_star)']

#then, for cols without upper limits 
for key in cols_to_split:
    temp_df = raw_df[key].str.split(' ', expand = True).copy()
    
    clean_df[key] = temp_df[0].replace(to_replace='', value=np.nan).astype(float)

    err_key = f'{key}_err'
    clean_df[err_key] = temp_df[2].replace(to_replace='', value=np.nan).astype(float)


cols_to_split = ['M_B', 'M_4.5']

#then, for cols with random appended strings in err cols (refs) 
for key in cols_to_split:
    temp_df = raw_df[key].str.split(' ', expand = True).copy()
    
    clean_df[key] = temp_df[0].astype(float)

    err_key = f'{key}_err'
    err_temp_df = temp_df[2].str.split('^', expand = True)
    clean_df[err_key] = err_temp_df[0].replace(to_replace='', value=np.nan).astype(float)

cols_to_split = ['B-V']

#then, for cols with random appended strings in value cols (refs) 
for key in cols_to_split:
    temp_df = raw_df[key].str.split(' ', expand = True).copy()
    value_temp_df = temp_df[0].str.split('^', expand = True)
    clean_df[key] = value_temp_df[0].replace(to_replace='', value=np.nan).astype(float)

    err_key = f'{key}_err'
    err_temp_df = temp_df[2].str.split('^', expand = True)
    clean_df[err_key] = err_temp_df[0].replace(to_replace='', value=np.nan).astype(float)



Fix RA + DEC to decimal degrees 

In [132]:
from astropy.coordinates import SkyCoord
import astropy.units as u 

In [142]:
sc = SkyCoord(clean_df['RA'], clean_df['DEC'], unit=(u.hourangle, u.deg))
clean_df['RA'] = sc.ra.value
clean_df['DEC'] = sc.dec.value

In [143]:
clean_df.to_csv('cleaned/table1.csv', index=False)

# Table 3

In [110]:
raw_df = pd.read_csv('data/apj433966t3_ascii.txt', header = None, skipfooter = 1, skiprows = 8, sep = '\t+')
columns = ['Ion', 'UGC 521 A',  'UGC 695 E', 'UGC 1056 A', 'UGC 1056 B', 'UGC 1176 A', 
           'NGC 784 A', 'NGC 784 B', 'UGC 2716 A', 'KKH 037 A', 'NGC 2537 A']
raw_df.columns = columns


/var/folders/g_/kxb_65m159995chtyg4q96440000gn/T/ipykernel_74613/287672149.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  raw_df = pd.read_csv('data/apj433966t3_ascii.txt', header = None, skipfooter = 1, skiprows = 8, sep = '\t+')


In [116]:
clean_df = raw_df[['Ion']].copy()

# split +/-
for key in columns[1:]:
    temp_df = raw_df[key].str.split(' ', expand = True).copy()
    
    clean_df[key] = temp_df[0].replace(to_replace='', value=np.nan).astype(float)

    err_key = f'{key} err'
    clean_df[err_key] = temp_df[2].replace(to_replace='', value=np.nan).astype(float)


In [118]:
clean_df.to_csv('cleaned/table3_test.csv', index=False)

# Table 4

In [145]:
raw_df = pd.read_csv('data/apj433966t4_ascii.txt', header = None, skipfooter = 1, skiprows = 6, sep = '\t+')
columns = ['Galaxy','Halpha Region','t_2', 't_3', 'O+/H+', 'O++/H+', 'O/H', '12 + log(O/H)', 'N+/H+', 'log(N/O)', 'N/H']
raw_df.columns = columns

/var/folders/g_/kxb_65m159995chtyg4q96440000gn/T/ipykernel_74613/724386464.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  raw_df = pd.read_csv('data/apj433966t4_ascii.txt', header = None, skipfooter = 1, skiprows = 6, sep = '\t+')


In [151]:
clean_df = raw_df[['Galaxy', 'Halpha Region']].copy()

# split +/-
for key in columns[2:]:
    temp_df = raw_df[key].str.split(' ', expand = True).copy()
    
    clean_df[key] = temp_df[0].replace(to_replace='', value=np.nan).astype(float)

    err_key = f'{key}_err'
    clean_df[err_key] = temp_df[2].replace(to_replace='', value=np.nan).astype(float)

In [190]:
table1 = pd.read_csv('cleaned/table1.csv')
clean_df_with_mass = clean_df.set_index('Galaxy').join(table1[['Galaxy', 'log(M_star)', 'log(M_star)_err']].set_index('Galaxy'), how='left')
clean_df_with_mass.reset_index(inplace=True)

In [194]:
clean_df_with_mass.to_csv('cleaned/table4_with_mass.csv', index=False)

# Formatted Table 3 crossmatch with Table 4

In [16]:
t4 = pd.read_csv('cleaned/table4_with_mass.csv')

In [30]:
data = pd.read_csv('data/formatted_table3.csv', sep = ',')
#data.rename(columns={'Unnamed: 0': 'Galaxy'}, inplace=True) #to rename col, not needed

# 'Unnamed: 0' is the first col of formatted table 3 (bc colname was left blank)
# this line splits that col into three cols e.g. 'UGC5272A' --> 'UGC' '5272' 'A'
# or 'CGCG269-049C' --> 'CGCG' '269-049' 'C'
name_minidf = data['Unnamed: 0'].str.extract('([A-Za-z]+)(\d+\-?\d*)([A-Za-z]*)', expand = True)

data['Galaxy'] = name_minidf[0] + ' ' + name_minidf[1].astype(str)
data['Halpha Region'] = name_minidf[2]
data.drop(columns='Unnamed: 0', inplace=True) 

In [34]:
merged_df= pd.merge(t4, data,  how='left', left_on=['Galaxy','Halpha Region'], right_on = ['Galaxy','Halpha Region'])
merged_df.to_csv('cleaned/combined_tables.csv', index=False)

In [38]:
merged_df.head(10)

,Galaxy,Halpha Region,t_2,t_2_err,t_3,t_3_err,O+/H+,O+/H+_err,O++/H+,O++/H+_err,...,[SII]6717,u_[SII]6717,[SII]6731,u_[SII]6731,c(Hbeta),u_c(Hbeta),F(Hbeta),u_F(Hbeta),EW(Hbeta),EW(Halpha)
0,CGCG 269-049,A,14400.0,500.0,17100.0,500.0,1.00,0.11,1.95,0.11,...,0.064,0.002,0.046,0.002,0.08,0.01,29.10,0.60,81.30,434.0
1,IC 559,A,12000.0,1400.0,11500.0,1300.0,5.63,2.31,6.18,2.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NGC 2537,A,10900.0,600.0,9700.0,500.0,8.85,1.94,8.56,1.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NGC 2537,B,11800.0,900.0,11200.0,900.0,5.26,1.48,4.23,0.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NGC 2552,A,11400.0,500.0,10400.0,500.0,5.27,0.88,8.93,1.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NGC 3738,A,12100.0,900.0,11800.0,800.0,4.98,1.27,6.00,1.21,...,0.320,0.010,0.230,0.010,0.04,0.01,58.70,1.20,35.20,183.0
6,NGC 3738,B,12500.0,1400.0,12500.0,1400.0,5.36,2.15,5.39,1.67,...,0.290,0.010,0.210,0.010,0.19,0.01,24.80,0.50,23.40,121.0
7,NGC 3741,A,13700.0,500.0,15200.0,400.0,1.81,0.21,2.93,0.20,...,0.122,0.002,0.088,0.002,0.10,0.01,45.00,0.90,59.90,330.0
8,NGC 4163,A,14800.0,2000.0,18200.0,2500.0,3.28,1.34,0.34,0.09,...,0.350,0.020,0.230,0.020,0.10,0.01,5.24,0.12,9.21,40.9
9,NGC 4449,C,11500.0,900.0,10600.0,800.0,6.92,1.99,6.69,1.59,...,0.350,0.010,0.250,0.010,0.14,0.01,54.80,1.10,119.00,437.0


# Matching with table 1

In [50]:
t1 = pd.read_csv('cleaned/table1.csv')
t14_merged = pd.merge(t1, t4, how = 'left', on="Galaxy")

In [52]:
merged_all = pd.merge(t14_merged, data,  how='left', left_on=['Galaxy','Halpha Region'], right_on = ['Galaxy','Halpha Region'])
merged_all.to_csv('cleaned/combined_tables.csv', index=False)